# Exploratory Data Analysis (EDA) — NYC Yellow Taxi 2024

## Introduction

In this section, we will perform an exploratory data analysis (EDA) on the full 12-month NYC Yellow Taxi Trip dataset, now fully loaded into MySQL (~41M records). The goal is to extract key insights and trends from the data, including:

- Total trips per month
- Average fare and tip analysis
- Payment type trends

## Why we are creating indexes

Since the dataset is large (~41 million rows), performing aggregation queries directly on the raw table can be very slow. To optimize query performance for our EDA, we will create indexes on the columns that are most frequently used in `GROUP BY`, `WHERE`, and `ORDER BY` clauses:

- `pu_location_id` → used for geospatial and pickup analysis
- `do_location_id` → used for dropoff analysis
- `tpep_pickup_datetime` → used to group trips by month and time

Creating these indexes will significantly improve the speed of our queries and make the analysis more efficient and interactive.

**Note:** The index creation step is safe to run multiple times — if an index already exists, MySQL will report it and skip re-creating it.


In [ ]:
# Create indexes to optimize EDA queries

import mysql.connector
from dotenv import load_dotenv
import os

# Load env variables
load_dotenv()

# MySQL connection config
MYSQL_CONFIG = {
    'user': os.getenv('MYSQL_USER'),
    'password': os.getenv('MYSQL_PASSWORD'),
    'host': os.getenv('MYSQL_HOST'),
    'database': os.getenv('MYSQL_DATABASE'),
    'port': int(os.getenv('MYSQL_PORT'))
}

# Connect to MySQL
cnx = mysql.connector.connect(**MYSQL_CONFIG)
cursor = cnx.cursor()

# List of index creation statements
index_statements = [
    "CREATE INDEX idx_pu_location_id ON yellow_taxi_trips(pu_location_id);",
    "CREATE INDEX idx_do_location_id ON yellow_taxi_trips(do_location_id);",
    "CREATE INDEX idx_tpep_pickup_datetime ON yellow_taxi_trips(tpep_pickup_datetime);"
]

# Execute index creation
for stmt in index_statements:
    try:
        print(f"Creating index: {stmt}")
        cursor.execute(stmt)
        cnx.commit()
        print("✅ Index created.")
    except mysql.connector.Error as err:
        # If index already exists, MySQL will throw an error → we catch it
        print(f"⚠️  Could not create index: {err.msg}")

# Close connection
cursor.close()
cnx.close()


Creating index: CREATE INDEX idx_pu_location_id ON yellow_taxi_trips(pu_location_id);
